In [1]:
# -*- coding: UTF-8 -*-
import tensorflow as tf
from sklearn.metrics import roc_curve
from sklearn.metrics import auc
from matplotlib import pyplot as plt
from Preprocess import label_for_nn

In [2]:
benign_path = '/home/user01/Desktop/DDSM/4_benign_Pleo_10_ROI_subpic'#input('良性子图路径')#
cancer_path = '/home/user01/Desktop/DDSM/4_cancer_Pleo_10_ROI_subpic'#input('恶性子图路径')#
saver_path = '/home/user01/Desktop/DDSM/model/AlexNet_v1'#input('模型保存路径')#

In [3]:
size = 128
train_time = 100
batch_size = 25
learning_rate = 6e-5
dropout_rate = 0.5

In [4]:
def get_trainsets_and_testsets():
    (train_x ,train_y),(test_x,test_y) = label_for_nn(benign_path,cancer_path,scale=0.2,kind='RGB')
    train_x = train_x.reshape(train_x.shape[0],size,size,3).astype('float32')/255.0
    test_x = test_x.reshape(test_x.shape[0],size,size,3).astype('float32')/255.0
    return train_x , test_x ,train_y ,test_y

In [5]:
def Weight_variable(shape,name):
    initial = tf.truncated_normal(shape,stddev=0.05)
    return tf.Variable(initial,name=name)

In [6]:
def biases_variable(shape,name):
    initial = tf.constant(0.0,shape=shape)
    return tf.Variable(initial,name=name)

In [7]:
def conv2d_P(x,W,s):
    return tf.nn.conv2d(x,W,strides=[1,s,s,1],padding='SAME')

In [8]:
def conv2d(x,W,s):
    return tf.nn.conv2d(x,W,strides=[1,s,s,1],padding='VALID')

In [9]:
def max_pooling2x2(x):
    return tf.nn.max_pool(x,ksize=[1,3,3,1],strides=[1,2,2,1],padding='VALID')

In [10]:
def cnn_train():
    with tf.name_scope('inputs'):
        x = tf.placeholder(tf.float32, [None, size, size, 3])
        y = tf.placeholder(tf.float32, [None, 2])
        keep_prob = tf.placeholder(tf.float32)
    # conv1#
    with tf.name_scope('layer_1'):
        with tf.name_scope('weight_1'):
            w_conv1 = Weight_variable([11, 11, 3, 64],'w_1') # 128
        with tf.name_scope('biases_1'):
            b_conv1 = biases_variable([64],'b_1')
        with tf.name_scope('conv_1'):
            h_conv1 = tf.nn.relu(conv2d_P(x, w_conv1, 4) + b_conv1)
        with tf.name_scope('pool_1'):
            h_pool1 = max_pooling2x2(h_conv1)
    #conv2
    with tf.name_scope('layer_2'):
        with tf.name_scope('weight_2'):
            w_conv2 = Weight_variable([5, 5, 64, 192],'w_2')  # 43-5+1=39*39*32
        with tf.name_scope('biases_2'):
            b_conv2 = biases_variable([192],'b_2')
        with tf.name_scope('conv_2'):
            h_conv2 = tf.nn.relu(conv2d_P(h_pool1, w_conv2, 1) + b_conv2)
        with tf.name_scope('pool_2'):
            h_pool2 = max_pooling2x2(h_conv2)
    #conv3
    with tf.name_scope('layer_3'):
        with tf.name_scope('weight_3'):
            w_conv3 = Weight_variable([3, 3, 192, 384],'w_3')  # 13-5+1 = 9*9*64
        with tf.name_scope('biases_3'):
            b_conv3 = biases_variable([384],'b_3')
        with tf.name_scope('conv_3'):
            h_conv3 = tf.nn.relu(conv2d_P(h_pool2,w_conv3, 1) +b_conv3)
    #conv4
    with tf.name_scope('layer_4'):
        with tf.name_scope('weight_4'):
            w_conv4 = Weight_variable([3, 3, 384, 256],'w_4')
        with tf.name_scope('biases_4'):
            b_conv4 = biases_variable([256],'b_4')
        with tf.name_scope('conv_4'):
            h_conv4 = tf.nn.relu(conv2d_P(h_conv3, w_conv4, 1) + b_conv4)
    #conv5
    with tf.name_scope('layer_5'):
        with tf.name_scope('weight_5'):
            w_conv5 = Weight_variable([3, 3, 256, 256],'w_5')
        with tf.name_scope('biases_5'):
            b_conv5 = biases_variable([256],'b_5')
        with tf.name_scope('conv_5'):
            h_conv5 = tf.nn.relu(conv2d_P(h_conv4,w_conv5, 1) + b_conv5)
        with tf.name_scope('pool_3'):
            h_pool3 = max_pooling2x2(h_conv5)
    # func1#
    with tf.name_scope('layer_6'):
        with tf.name_scope('weight_6'):
            w_conv6 = Weight_variable([3 * 3 * 256, 4096],'w_6')
        with tf.name_scope('biases_6'):
            b_conv6 = biases_variable([4096],'b_6')
        with tf.name_scope('fc_1'):
            pool5_flat = tf.reshape(h_pool3, [-1,3 * 3 * 256])
            fc1 = tf.nn.relu(tf.matmul(pool5_flat, w_conv6) + b_conv6)
            fc1_drop = tf.nn.dropout(fc1, keep_prob)
    # func2#
    with tf.name_scope('layer_7'):
        with tf.name_scope('weight_7'):
            w_conv7 = Weight_variable([4096, 4096],'w_7')
        with tf.name_scope('biases_7'):
            b_conv7 = biases_variable([4096],'b_7')
        with tf.name_scope('fc_2'):
            fc2 = tf.nn.relu(tf.matmul(fc1_drop, w_conv7) + b_conv7)
            fc2_drop = tf.nn.dropout(fc2, keep_prob)
    # func3#
    with tf.name_scope('layer_8'):
        with tf.name_scope('weight_8'):
            w_conv8 = Weight_variable([4096, 2],'w_8')
        with tf.name_scope('biases_8'):
            b_conv8 = biases_variable([2],'b_8')
        with tf.name_scope('fc_3'):
            y_pre = tf.nn.softmax(tf.matmul(fc2_drop, w_conv8) + b_conv8)
    with tf.name_scope('cross_entropy'):
        cross_entropy = tf.reduce_mean(-tf.reduce_sum(y * tf.log(y_pre), reduction_indices=[1]))
    with tf.name_scope('trian'):
        train_step = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cross_entropy)
    y_pre_label = tf.argmax(y_pre,1)
    y_label = tf.argmax(y,1)
    a = tf.add(y_pre_label,1)
    b = tf.add(y_label,3)
    c = tf.multiply(a,b)
    tp = tf.reduce_sum(tf.cast(tf.equal(c,8),tf.float32))
    fp = tf.reduce_sum(tf.cast(tf.equal(c,6),tf.float32))
    tn = tf.reduce_sum(tf.cast(tf.equal(c,3),tf.float32))
    fn = tf.reduce_sum(tf.cast(tf.equal(c,4),tf.float32))
    tpr = tp/(tp+fn)
    tnr = tn/(tn+fp)
    tf.summary.scalar('Sensitivity', tpr)
    tf.summary.scalar('Specificity', tnr)
    correct_l1 = tf.equal(y_pre_label,y_label)
    accuracy = tf.reduce_mean(tf.cast(correct_l1,tf.float32))
    tf.summary.scalar('loss',cross_entropy)
    tf.summary.scalar('accuracy',accuracy)
    merged_summary_op = tf.summary.merge_all()
    init = tf.global_variables_initializer()
    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True
    saver = tf.train.Saver({'w_1': w_conv1, 'b_1': b_conv1,
                            'w_2': w_conv2, 'b_2': b_conv2,
                            'w_3': w_conv3, 'b_3': b_conv3,
                            'w_4': w_conv4, 'b_4': b_conv4,
                            'w_5': w_conv5, 'b_5': b_conv5,
                            'w_6': w_conv6, 'b_6': b_conv6,
                            'w_7': w_conv7, 'b_7': b_conv7,
                            'w_8': w_conv8, 'b_8': b_conv8
                            })
    with tf.Session(config=config) as sess:
        sess.run(init)
        writer = tf.summary.FileWriter(saver_path, sess.graph)
        sum_test_acc = 0
        train_x,test_x,train_y,test_y=get_trainsets_and_testsets()#获取训练集和测试集
        num_batch = len(train_x)//batch_size
        for n in range(1,train_time+1):
            sum_train_acc = 0
            av_train_loss = 0
            for i in range(num_batch):#训练
                batch_x = train_x[i*batch_size : (i+1)*batch_size]
                batch_y = train_y[i*batch_size : (i+1)*batch_size]
                _,loss,train_acc= sess.run([train_step,cross_entropy,accuracy],feed_dict={x:batch_x,y:batch_y,keep_prob:dropout_rate})
                sum_train_acc += train_acc
                av_train_loss += loss
            av_train_loss /= num_batch
            sum_train_acc /= num_batch
            test_acc, pre, test_loss = sess.run([accuracy, y_pre, cross_entropy],feed_dict={x: test_x, y: test_y, keep_prob: 1.0})
            if n>30:
                sum_test_acc += test_acc
            #验证
            print('{0:3d} train_loss: {1:>10.9f} test_loss: {2:>10.9f} train_accuracy:{3:>10.9f} test_accuracy:{4:>10.9f}'.format(n,av_train_loss,test_loss,sum_train_acc,test_acc))
            summary = sess.run(merged_summary_op,feed_dict={x:test_x,y:test_y,keep_prob:1.0})
            writer.add_summary(summary=summary, global_step=n)
        #ROC曲线
        y_,y_p= sess.run([y_label,y_pre],feed_dict={x: test_x, y: test_y, keep_prob: 1.0})
        temp = []
        for i in range(len(y_p)):
            temp.append(y_p[i][1])
        fpr,tpr,thresholds = roc_curve(y_,temp)
        n_auc = auc(fpr,tpr)
        plt.plot(fpr,tpr,linewidth=2)
        plt.plot([0,1],[0,1],'k--')
        plt.axis([0,1,0,1])
        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        text = 'AUC:'+str(n_auc)[0:5]
        plt.text(0.7,0.2,text,fontdict={'size':'12','color':'b'})
        saver.save(sess,saver_path+'/AlexNet')
        print(sum_test_acc/(train_time-30))
        plt.savefig(saver_path+'/AlexNet.png')

In [11]:
cnn_train()

  1 train_loss: 0.732424182 test_loss: 0.687576652 train_accuracy:0.488000003 test_accuracy:0.473684222
  2 train_loss: 0.676422912 test_loss: 0.650260568 train_accuracy:0.591999990 test_accuracy:0.649122834
  3 train_loss: 0.633758879 test_loss: 0.628666699 train_accuracy:0.612000006 test_accuracy:0.684210539
  4 train_loss: 0.608446717 test_loss: 0.604785144 train_accuracy:0.676000005 test_accuracy:0.736842096
  5 train_loss: 0.572279865 test_loss: 0.576753914 train_accuracy:0.684000009 test_accuracy:0.736842096
  6 train_loss: 0.536500818 test_loss: 0.546446323 train_accuracy:0.700000006 test_accuracy:0.736842096
  7 train_loss: 0.484986770 test_loss: 0.522342384 train_accuracy:0.772000003 test_accuracy:0.789473712
  8 train_loss: 0.460751098 test_loss: 0.509600878 train_accuracy:0.751999998 test_accuracy:0.789473712
  9 train_loss: 0.456349885 test_loss: 0.504718125 train_accuracy:0.764000005 test_accuracy:0.807017565
 10 train_loss: 0.427312273 test_loss: 0.496698976 train_accurac

 80 train_loss: 0.000324927 test_loss: 0.243590251 train_accuracy:1.000000000 test_accuracy:0.929824591
 81 train_loss: 0.000470511 test_loss: 0.235026315 train_accuracy:1.000000000 test_accuracy:0.929824591
 82 train_loss: 0.000653145 test_loss: 0.227148712 train_accuracy:1.000000000 test_accuracy:0.929824591
 83 train_loss: 0.000342440 test_loss: 0.266106933 train_accuracy:1.000000000 test_accuracy:0.912280679
 84 train_loss: 0.000413061 test_loss: 0.280105799 train_accuracy:1.000000000 test_accuracy:0.912280679
 85 train_loss: 0.000469214 test_loss: 0.233756334 train_accuracy:1.000000000 test_accuracy:0.929824591
 86 train_loss: 0.000338614 test_loss: 0.245929658 train_accuracy:1.000000000 test_accuracy:0.929824591
 87 train_loss: 0.000378845 test_loss: 0.284429491 train_accuracy:1.000000000 test_accuracy:0.929824591
 88 train_loss: 0.000393488 test_loss: 0.256743729 train_accuracy:1.000000000 test_accuracy:0.929824591
 89 train_loss: 0.000332163 test_loss: 0.261540741 train_accurac

AttributeError: 'module' object has no attribute 'to_rgba'